In [ ]:
# pip install transformers

# https://github.com/00ddya3/20232_DL/blob/74eba1147edae18e64b1b5367e05f58a2e002157/1030/BERT_Kor_movie_reviews_sentiment_fine_tuning_KcBERT.ipynb#L51
# 더 간단한 코드

In [ ]:
import transformers
transformers.__version__

'4.35.2'

In [ ]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys

print(os.getcwd())

/content


In [ ]:
import pandas as pd

train_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')
train_data_cl1 = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')
train_data_cl2 = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.hate.csv')

train_newstitle = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/train.news_title.txt', sep='\t', names=['news_title'])

# unlabeled_comments = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/unlabeled_comments.txt', sep='\t', names=['unlabeled_comments'])

dev_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/dev.hate.csv')
test_data = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/test.hate.no_label.csv')

test_newstitle = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/test.news_title.txt', sep='\t', names=['news_title'])

# , sep='\t', quoting=3

In [ ]:
# train_data.head()

is_NaN = train_newstitle.isnull()
print(is_NaN)
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = train_newstitle[row_has_NaN]
rows_with_NaN

      news_title
0          False
1          False
2          False
3          False
4          False
...          ...
7891       False
7892       False
7893       False
7894       False
7895       False

[7896 rows x 1 columns]


,news_title
1522,NaN
5574,NaN
7150,NaN


In [ ]:
train_data_titled = train_data.copy()
train_data_titled['comments'] = train_data_titled['comments'] + train_newstitle['news_title']

In [ ]:
print(train_data['comments'][1])
print(train_data_titled['comments'][1])

train_data_titled.head()

....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ
....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을 미처 알지못했네요ㅠ'연중' 故 전미선, 생전 마지막 미공개 인터뷰…환하게 웃는 모습 '먹먹'[종합]


,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데'아스달 연대기' 장동...",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [ ]:
LABEL_DIC = {
    'none': 0,
    'offensive': 1,
    'hate': 2,
}

train_data['label'] = train_data['label'].map(lambda x: LABEL_DIC[x])
train_data_cl1['label'] = train_data_cl1['label'].map(lambda x: LABEL_DIC[x])
train_data_cl2['label'] = train_data_cl2['label'].map(lambda x: LABEL_DIC[x])
train_data_titled['label'] = train_data['label']

dev_data['label'] = dev_data['label'].map(lambda x: LABEL_DIC[x])

In [ ]:
train_data_titled.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,2
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",2
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데'아스달 연대기' 장동...",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,2


In [ ]:
# 데이터 정제

import string

## remove basic punctuation
def remove_punc(text):
    text_nopunc = "".join([char for char in text if char not in string.punctuation])
    return text_nopunc

train_data_cl1['comments'] = train_data_cl1['comments'].apply(lambda x: remove_punc(x))

# train_data['news_title'] = train_data['news_title'].apply(lambda x: remove_punc(x))

dev_data['comments'] = dev_data['comments'].apply(lambda x: remove_punc(x))

test_data['comments'] = test_data['comments'].apply(lambda x: remove_punc(x))

In [ ]:
# train_data_cl1['comments']

In [ ]:
import re

# remove all punctuations except korean, english, and number
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[a-zA-Z]', '', text)
    text = re.sub('[ㄱ-ㅎㅏ-ㅣ]+', '', text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', text)
    return text

train_data_cl2['comments'] = train_data_cl2['comments'].apply(cleanse)
# train['news_title'] = train['news_title'].apply(cleanse)

dev_data['comments'] = dev_data['comments'].apply(cleanse)

test_data['comments'] = test_data['comments'].apply(cleanse)

# train_data.head()

In [ ]:
# train_data_list = []

# for p, q, r in zip(train_data['comments'], train_data['clean_comments'], train_data['clean2_comments'])  :
#     data = []
#     data.append(q + q + r)
#     # data.append(str(label))

#     train_data_list.append(data)

# dev_data_list = []

# for q, label in zip(dev_data['comments'], dev_data['label'])  :
#     data = []
#     data.append(q)
#     data.append(str(label))

#     dev_data_list.append(data)

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', mask_token='<mask>')

# tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")

tokenizer = AutoTokenizer.from_pretrained('beomi/kcbert-base', bos_token='</s>', eos_token='</s>', pad_token='<pad>', mask_token='<mask>')

In [ ]:
# print(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
# print(tokenizer.tokenize("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))
# tokenizer.decode(tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"))

# for elem in tokenizer.encode("보는내내 그대로 들어맞는 예측 카리스마 없는 악역"):
#   print(tokenizer.decode(elem))

# print(tokenizer.tokenize("전율을 일으키는 영화. 다시 보고싶은 영화"))

# print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"))

# for elem in tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화"):
#   print(tokenizer.decode(elem))

# for elem in tokenizer.encode("happy birthday~!"):
#   print(tokenizer.decode(elem))

# print(tokenizer.decode(0))
# print(tokenizer.decode(1))
# print(tokenizer.decode(2))
# print(tokenizer.decode(3))
# print(tokenizer.decode(4))

In [ ]:
max_seq_len = 140

In [ ]:
# encoded_result = tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=max_seq_len, padding='max_length')
# print(encoded_result)
# print('길이 :', len(encoded_result))

# print(tokenizer.encode("전율을 일으키는 영화. 다시 보고싶은 영화", max_length=140))

In [ ]:
def convert_examples_to_features(comments, max_seq_len, tokenizer):

    input_ids = []

    for sent1 in tqdm(comments, total = len(comments)):
        # bos_token = [tokenizer.bos_token]
        # eos_token = [tokenizer.eos_token]
        # sent1_tokens = bos_token + tokenizer.tokenize(sent1) + eos_token + ['']

        sent1_tokens = [tokenizer.bos_token] + tokenizer.tokenize(sent1) + [tokenizer.eos_token]

        tokens = sent1_tokens
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)

    input_ids = np.array(input_ids, dtype=int)

    return input_ids

In [ ]:
train_data.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,2
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",2
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,2


In [ ]:
new_train_data = pd.concat([train_data, train_data_cl1, train_data_cl2, train_data_titled])

new_train_data.shape

(31584, 2)

In [ ]:
# X_train 정의
# 자꾸 에러난다!

X_train = convert_examples_to_features(new_train_data['comments'], max_seq_len = max_seq_len, tokenizer = tokenizer)

 80%|███████▉  | 25210/31584 [00:05<00:01, 4778.63it/s]


TypeError: ignored

In [ ]:
input_id = X_train[0]

print('단어에 대한 정수 인코딩 :', input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [30000    11  8979 16099 17784 20383    12  2170  9168   591  1293  4384
 26536  4113   595 29540 12020 16099  4227  8007 13999  7972  9229  8249
 15042  4282  4011    17    17    17    17 30000 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001]
각 인코딩의 길이 : 140
정수 인코딩 복원 : </s> ( 현재 호텔주인 심정 ) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속 추모받네.... </s> <pad> <pad> <pad> <pad> <pad> <pad>

In [ ]:
X_dev = convert_examples_to_features(dev_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 471/471 [00:00<00:00, 3268.63it/s]


In [ ]:
# 최대 길이: 128
input_id = X_dev[0]

print('단어에 대한 정수 인코딩 :',input_id)
print('각 인코딩의 길이 :', len(input_id))
print('정수 인코딩 복원 :',tokenizer.decode(input_id))

단어에 대한 정수 인코딩 : [30000 26295  4184  9349  4496  4057  9246  9918  2881  4375 23645  7968
 12307  4020 30000 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001 30001
 30001 30001 30001 30001 30001 30001 30001 30001]
각 인코딩의 길이 : 140
정수 인코딩 복원 : </s> 송중기 시대극은 믿고본다 첫회 신선하고 좋았다 </s> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

In [ ]:
X_test = convert_examples_to_features(test_data['comments'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 974/974 [00:00<00:00, 5534.83it/s]


In [ ]:
train_label = new_train_data['label'].tolist()
dev_label = dev_data['label'].tolist()
# test_label = test_data['label'].tolist()

# train_label

In [ ]:
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)

y_train = idx_encode.transform(train_label) # 주어진 고유한 정수로 변환
y_dev = idx_encode.transform(dev_label) # 고유한 정수로 변환
# y_test = idx_encode.transform(test_label) # 고유한 정수로 변환

label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
idx_label = {value: key for key, value in label_idx.items()}
print(label_idx)
print(idx_label)

{0: 0, 1: 1, 2: 2}
{0: 0, 1: 1, 2: 2}


In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification

model = TFBertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels=3, from_pt=True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

In [ ]:
max_seq_len = 140

In [ ]:
# input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
# outputs = model([input_ids_layer])

# print(outputs)
# print(outputs[0])
# print(outputs[1])
# print(outputs[0][:, -1])

In [ ]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

TPU 사용법 : https://wikidocs.net/119990

In [ ]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

In [ ]:
with strategy.scope():
  model = TFBertForSequenceClassification.from_pretrained("beomi/kcbert-base", num_labels = 3, from_pt = True, hidden_dropout_prob = 0.3)

  # hidden_dropout_prob = 0.3

  # model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2", num_labels=3)

  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-6)
  loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])


# https://huggingface.co/transformers/v2.10.0/model_doc/bert.html#tfbertforsequenceclassification

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.utils import to_categorical

y_train_encoded = to_categorical(y_train)
y_dev_encoded = to_categorical(y_dev)

In [ ]:
early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta = 0.001,
    patience = 2)

history = model.fit(X_train, y_train_encoded, epochs = 15, batch_size = 64, validation_data=(X_dev, y_dev_encoded), callbacks = [early_stopping])

# max = 0.6858 / 0.7070
# 6391 (2e-6)
# 0.6539
# 0.6815

Epoch 1/15
371/371 [==============================] - 104s 131ms/step - loss: 0.5457 - accuracy: 0.5518 - val_loss: 0.5585 - val_accuracy: 0.5945
Epoch 2/15
371/371 [==============================] - 44s 118ms/step - loss: 0.4561 - accuracy: 0.6562 - val_loss: 0.5867 - val_accuracy: 0.5817
Epoch 3/15
371/371 [==============================] - 44s 118ms/step - loss: 0.4166 - accuracy: 0.7013 - val_loss: 0.5371 - val_accuracy: 0.6433
Epoch 4/15
371/371 [==============================] - 43s 117ms/step - loss: 0.3765 - accuracy: 0.7402 - val_loss: 0.5453 - val_accuracy: 0.6518
Epoch 5/15
371/371 [==============================] - 43s 116ms/step - loss: 0.3363 - accuracy: 0.7779 - val_loss: 0.6382 - val_accuracy: 0.6348
Epoch 6/15
371/371 [==============================] - 43s 117ms/step - loss: 0.3046 - accuracy: 0.8050 - val_loss: 0.6969 - val_accuracy: 0.6008


In [ ]:
# 토큰화 된 상태
# X_test

In [ ]:
output = model.predict(X_test)

31/31 [==============================] - 8s 52ms/step


In [ ]:
output.logits

array([[-1.0084181,  0.8895212, -3.7338848],
       [ 3.9930847, -3.9595814, -4.42766  ],
       [ 3.7114484, -3.7615535, -4.8199534],
       ...,
       [ 3.705314 , -3.8529658, -4.71924  ],
       [-5.17873  , -2.7526445,  3.0078664],
       [ 3.8014896, -3.84498  , -4.705928 ]], dtype=float32)

In [ ]:
import torch

outputTensor = torch.Tensor(output.logits)

In [ ]:
# outputTensor = nn.Softmax(dim=1)(outputTensor)
pred = torch.argmax(outputTensor, dim=-1)

In [ ]:
pred

tensor([1, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 0, 1, 1, 2, 1, 0, 0, 1, 0, 1, 0, 1, 1,
        1, 0, 2, 0, 2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 1, 0, 1, 2, 1, 0, 2, 0, 0,
        0, 2, 2, 0, 0, 1, 0, 2, 1, 1, 0, 2, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 0, 1,
        0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 2, 0, 1, 2, 0, 2, 0, 1, 2,
        2, 0, 1, 0, 0, 0, 0, 1, 0, 0, 2, 0, 1, 0, 0, 0, 1, 0, 0, 2, 1, 1, 0, 0,
        0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0,
        0, 2, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 2, 1, 0, 1, 2, 1, 2, 1, 1, 1, 0,
        0, 1, 2, 1, 0, 0, 1, 1, 0, 1, 2, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 2, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 2, 0, 1, 0, 0, 1, 0, 0, 0,
        2, 0, 0, 1, 1, 1, 2, 2, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 0, 1, 0, 1, 2, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 2, 1, 1, 0, 0, 1, 1, 1,
        0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 2,

In [ ]:
test_label = [int(k.cpu()) for k in pred]
test_data['label'] = test_label

In [ ]:
test_data

,comments,label
0,그래도 조아해주는 팬들 많아서 좋겠다 니들은 온유가 안만져줌,1
1,둘다 넘 좋다행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다,0
...,...,...
969,대박 게스트 꼭 봐야징 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척 성형 전 니 얼굴 다 알고있다 순자처럼 된장냄새나게...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨 우리나라사람들은 분위기만 비슷하면 다 표...,0
972,입에 손가릭이 10개 있으니 징그럽다,2


In [ ]:
testreset = pd.read_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/test.hate.no_label.csv')

test_data['comments'] = testreset['comments']

In [ ]:
test_data

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,1
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2


In [ ]:
test_data.to_csv('/content/drive/MyDrive/GoogleML/korean-hate-speech-detection/submission/KcBERT_test.csv', index=None)